# 0. 라이브러리

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from haversine import haversine, Unit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import preprocessing as pp

from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

import feat_eng
import utils
import gc

from sklearn.neighbors import BallTree

import lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostRegressor
import optuna
import joblib

from geopy.distance import geodesic

import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [4]:
file_path = './data/'
train = pd.read_csv(os.path.join(file_path, "train.csv")).assign(_type='train')
test = pd.read_csv(os.path.join(file_path, 'test.csv')).assign(_type='test')

# 1. 전처리

### 중복 제거

In [5]:
print(f'중복 제거 전 train.shape = {train.shape}')
dup_train = train.drop('index', axis=1).duplicated()
train = train[~dup_train]
print(f'중복 제거 후 train.shape = {train.shape}')

df = pd.concat([train, test])
del train, test, dup_train
print(f'df.shape = {df.shape}')

중복 제거 전 train.shape = (1801228, 12)
중복 제거 후 train.shape = (1717611, 12)
df.shape = (1867783, 12)


# 2. Feature Engineering

### 1) `complex_id` 할당

In [6]:
class ComplexIdAssigner:
    def __init__(self, radius=2):
        self.radius = radius  # 반경 (km)
        self.tree = None
        self.df_not_neg1_unique = None
        self.complex_id_map = None

    def fit(self, df_train):
        # 학습 데이터에서 unique한 longitude, latitude로 새로운 complex_id 생성
        df_train = df_train.copy()
        df_train['complex_id'] = df_train.groupby(['latitude', 'longitude']).ngroup()
        self.complex_id_map = df_train[['latitude', 'longitude', 'complex_id']].drop_duplicates().reset_index(drop=True)

        # complex_id가 있는 데이터 추출
        df_not_neg1 = df_train[df_train['complex_id'] != -1]

        # unique한 complex_id 및 각 complex_id에 대한 최대 age 값 추출
        self.df_not_neg1_unique = df_not_neg1.groupby('complex_id', as_index=False).agg({
            'age': 'max',
            'latitude': 'first',
            'longitude': 'first'
        })

        # 위도와 경도를 라디안으로 변환
        self.df_not_neg1_unique['latitude_radians'] = np.radians(self.df_not_neg1_unique['latitude'])
        self.df_not_neg1_unique['longitude_radians'] = np.radians(self.df_not_neg1_unique['longitude'])

        # BallTree 구축
        coordinates = np.vstack((
            self.df_not_neg1_unique['latitude_radians'],
            self.df_not_neg1_unique['longitude_radians']
        )).T
        self.tree = BallTree(coordinates, metric='haversine')

        return self

    def transform(self, df_test):
        df_test = df_test.copy()
        # 학습 데이터의 complex_id를 기준으로 테스트 데이터에 complex_id 부여
        df_test = df_test.merge(self.complex_id_map, on=['latitude', 'longitude'], how='left')
        df_test['complex_id'] = df_test['complex_id'].fillna(-1).astype(int)

        # complex_id가 -1인 데이터 처리
        df_neg1 = df_test[df_test['complex_id'] == -1].reset_index(drop=True)

        if df_neg1.empty:
            return df_test

        # 위도와 경도를 라디안으로 변환
        df_neg1['latitude_radians'] = np.radians(df_neg1['latitude'])
        df_neg1['longitude_radians'] = np.radians(df_neg1['longitude'])

        # 좌표 추출
        query_coords = np.vstack((
            df_neg1['latitude_radians'],
            df_neg1['longitude_radians']
        )).T

        # 반지름을 라디안으로 변환
        radius_in_radians = self.radius / 6371  # 지구 반지름(km)

        # 2km 내의 이웃 찾기
        indices_array = self.tree.query_radius(query_coords, r=radius_in_radians)

        # 2km 내에 이웃이 없는 경우, 가장 가까운 이웃을 찾습니다.
        distances, nearest_indices = self.tree.query(query_coords, k=1)

        new_complex_ids = []
        for i in tqdm(range(len(df_neg1))):
            idxs = indices_array[i]
            if len(idxs) > 0:
                # 2km 내의 이웃이 있는 경우
                candidate_ages = self.df_not_neg1_unique.iloc[idxs]['age']
                age_diffs = abs(candidate_ages - df_neg1['age'].iloc[i])
                min_idx = idxs[age_diffs.argmin()]
            else:
                # 2km 내에 이웃이 없는 경우, 가장 가까운 이웃 사용
                min_idx = nearest_indices[i][0]
            new_complex_id = self.df_not_neg1_unique.iloc[min_idx]['complex_id']
            new_complex_ids.append(new_complex_id)

        # 새로운 complex_id를 df_neg1에 할당
        df_neg1['complex_id'] = new_complex_ids

        # complex_id가 -1인 부분을 업데이트
        df_test.loc[df_test['complex_id'] == -1, 'complex_id'] = df_neg1['complex_id'].values

        return df_test

    def fit_transform(self, df_train):
        self.fit(df_train)
        return self.transform(df_train)

In [7]:
# 학습 데이터와 테스트 데이터 분할
train_data = df[df['_type'] == 'train'].reset_index(drop=True)
test_data = df[df['_type'] == 'test'].reset_index(drop=True)

# ComplexIdAssigner 인스턴스 생성 및 학습
assigner = ComplexIdAssigner(radius=2)
assigner.fit(train_data)

# 학습 데이터에 transform 적용 (선택 사항)
train_data = assigner.transform(train_data)

# 테스트 데이터에 transform 적용
test_data = assigner.transform(test_data)

df = pd.concat([train_data, test_data], ignore_index=True)

100%|██████████| 1699/1699 [00:01<00:00, 1434.88it/s]


### 2) `max_deposit` 할당하기

- `complex_id`, `area_m2` 별 `deposit`의 '특정 통계값' 계산하기

In [8]:
class DepositStatCalculator:
    def __init__(self, stats=['max']):
        self.stats = stats
        self.stat_deposit = None
        self.complex_id_stats = {}

    def fit(self, df_train):
        # 학습 데이터에서 complex_id와 area_m2별로 지정된 통계값 계산
        groupby_cols = ['complex_id', 'area_m2']
        agg_dict = {'deposit': self.stats}
        self.stat_deposit = df_train.groupby(groupby_cols).agg(agg_dict).fillna(0).reset_index()

        # 컬럼 이름을 평탄화
        self.stat_deposit.columns = groupby_cols + [stat + '_deposit' for stat in self.stats]

        # 각 complex_id에 대한 area_m2와 통계값 저장
        for complex_id, group in self.stat_deposit.groupby('complex_id'):
            area_m2_array = group['area_m2'].values
            stats_values = group[[stat + '_deposit' for stat in self.stats]].values
            idx_sort = np.argsort(area_m2_array)
            area_m2_array = area_m2_array[idx_sort]
            stats_values = stats_values[idx_sort]
            self.complex_id_stats[complex_id] = (area_m2_array, stats_values)
        return self

    def transform(self, df):
        df = df.copy()
        # 계산된 통계값을 데이터프레임에 병합
        df = df.merge(self.stat_deposit, on=['complex_id', 'area_m2'], how='left')

        # 통계값이 누락된 경우(예: 테스트 데이터), 가장 가까운 area_m2의 통계값으로 대체
        for stat in self.stats:
            stat_col = f'{stat}_deposit'
            missing = df[stat_col].isnull()
            if missing.any():
                missing_df = df[missing]
                # 각 complex_id별로 처리
                for complex_id, group in missing_df.groupby('complex_id'):
                    if complex_id not in self.complex_id_stats:
                        # 해당 complex_id에 대한 통계값이 없는 경우
                        continue
                    area_m2_array, stats_values = self.complex_id_stats[complex_id]
                    area_m2_values = group['area_m2'].values
                    # 가장 가까운 area_m2 찾기
                    diffs = np.abs(area_m2_array[:, np.newaxis] - area_m2_values)
                    idxs_nearest = diffs.argmin(axis=0)
                    idxs_df = group.index
                    for i, idx_df in enumerate(idxs_df):
                        idx_nearest = idxs_nearest[i]
                        df.at[idx_df, stat_col] = stats_values[idx_nearest][self.stats.index(stat)]
        return df

    def fit_transform(self, df_train):
        self.fit(df_train)
        return self.transform(df_train)

In [9]:
%%time
# 학습 데이터와 테스트 데이터 분리
train_data = df[df['_type'] == 'train'].reset_index(drop=True)
test_data = df[df['_type'] == 'test'].reset_index(drop=True)

# DepositStatCalculator 인스턴스 생성 (그룹화할 컬럼과 계산할 통계값 지정)
deposit_calculator = DepositStatCalculator(stats=['max'])

# 학습 데이터에 대해 fit
train_data = deposit_calculator.fit_transform(train_data)

# 테스트 데이터에 transform 적용
test_data = deposit_calculator.transform(test_data)

df = pd.concat([train_data, test_data], ignore_index=True)

CPU times: user 11.2 s, sys: 691 ms, total: 11.9 s
Wall time: 11.6 s


### 이상거래 제거

- 중위수 기반 (modified Z-score)
- IQR 기반

일단은 진욱이형이 쓴 코드로 진행 ...

In [10]:
%%time
def remove_outliers(group):
    q1 = group['deposit'].quantile(0.25)
    q3 = group['deposit'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return group[(group['deposit'] >= lower_bound) & (group['deposit'] <= upper_bound)]

# 데이터 불러오기
train_data = df[df['_type'] == 'train'].reset_index(drop=True)
test_data = df[df['_type'] == 'test'].reset_index(drop=True)

train_data_cleaned = train_data.groupby(['complex_id', 'area_m2']).apply(remove_outliers).reset_index(drop=True)
print(f'제거된 데이터 개수: {len(train_data) - len(train_data_cleaned)}')

제거된 데이터 개수: 74777
CPU times: user 2min, sys: 2.55 s, total: 2min 2s
Wall time: 2min 2s


In [11]:
df = pd.concat([train_data_cleaned, test_data])
print(f"df.shape = {df.shape}")

df.shape = (1793006, 14)


### 클러스터링

In [13]:
import dr_clust

data = df.groupby(['latitude', 'longitude']).agg({'max_deposit': 'mean'}).reset_index()
print(data.shape)

(18676, 3)


In [14]:
%%time
selected_columns = ['latitude', 'longitude', 'max_deposit']

# 데이터 전처리 함수 호출
processed_data = dr_clust.select_and_preprocess_data(
    data=data,
    selected_columns=selected_columns,
    scaling_method='standard',  # 'standard' 또는 'minmax', 'robust'
    sample_size=None,           # 샘플링하지 않음
    random_state=42
)

# 차원 축소 알고리즘 선택 및 파라미터 설정
dr_method = 'UMAP'
dr_params = {'n_components': 2}

# 차원 축소 적용
dr_model, reduced_data, dr_used_params = dr_clust.apply_dimensionality_reduction(
    method=dr_method,
    data=processed_data,
    tune_hyperparameters=False,
    custom_params=dr_params,
    random_state=42
)

# 클러스터링 알고리즘 선택 및 파라미터 설정
cl_method = 'KMeans'
cl_params = {'n_clusters': 1500}

# 클러스터링 적용
model, labels, cl_used_params = dr_clust.apply_clustering(
    method=cl_method,
    data=reduced_data,
    tune_hyperparameters=False,
    custom_params=cl_params,
    random_state=42
)

data['cluster_labels'] = labels
df = df.merge(data[['latitude', 'longitude', 'cluster_labels']],
         on=['latitude', 'longitude'], how='left')

CPU times: user 1min 19s, sys: 6.48 s, total: 1min 26s
Wall time: 48.5 s


### ARIMA

In [15]:
# deposit_per_area, contract_year, contract_month 정의
df['deposit_per_area'] = df['deposit'] / df['area_m2']

df['contract_year'] = df['contract_year_month'].astype(str).str[:4]
df['contract_month'] = df['contract_year_month'].astype(str).str[4:6]

# Cluster별 contract_year에 따른 deposit_per_area의 평균
df = df.merge(df.groupby(['cluster_labels', 'contract_year'])['deposit_per_area'].mean().reset_index().rename({'deposit_per_area': 'mean_deposit_per_area_year'}, axis=1),
              on=['cluster_labels', 'contract_year'], how='left')

# df['contract_year_month'] = pd.to_datetime(df['contract_year_month'], format='%Y%m')

train_data = df[df['_type']=='train'].reset_index(drop=True)

# cluster_labels와 contract_year로 중복 제거
data = train_data[['cluster_labels', 'contract_year', 'mean_deposit_per_area_year']].drop_duplicates().reset_index(drop=True)
data = data.sort_values(by=['cluster_labels', 'contract_year'])

In [16]:
# 각 cluster labels별로 ARIMA 모델로 2024년 예측
predictions = []

for cluster_label in tqdm(data['cluster_labels'].unique()):
    # 각 cluster_label에 대한 19~23년 데이터 선택
    cluster_data = data[data['cluster_labels']==cluster_label]

    # 데이터 충분성 체크: ARIMA 모델 학습을 위해 일정 개수 이상의 데이터가 필요
    if len(cluster_data) < 3:
        print(f"cluster_labels {cluster_label}: 데이터가 부족하여 ARIMA 모델을 학습할 수 없습니다.")
        continue

    # ARIMA 모델 적합
    try:
        model = auto_arima(cluster_data['mean_deposit_per_area_year'], start_p=1, start_q=1, max_p=3, max_q=3, seasonal=False, trace=False,
                           error_action='ignore', suppress_warnings=True)
        fitted_model = ARIMA(cluster_data['mean_deposit_per_area_year'], order = model.order).fit()
        # 24년에 대한 예측(단일 예측값)
        forecast = fitted_model.get_forecast(steps=1).predicted_mean
        # 결과 저장 (cluster_labels과 예측값)
        predictions.append({'cluster': cluster_label, 'pred_deposit': forecast.values[0], 'year': 2024})
    except Exception as e:
        print(f"cluster_labels {cluster_label}: ARIMA 모델 학습 중 오류 발생 - {e}")

result_df = pd.DataFrame(predictions)
result_df.columns = ['cluster_labels', 'mean_deposit_per_area_year', 'contract_year']

 13%|█▎        | 196/1500 [01:30<08:54,  2.44it/s]

cluster_labels 196: 데이터가 부족하여 ARIMA 모델을 학습할 수 없습니다.


 50%|█████     | 755/1500 [05:47<05:45,  2.16it/s]

cluster_labels 755: 데이터가 부족하여 ARIMA 모델을 학습할 수 없습니다.


 53%|█████▎    | 788/1500 [06:02<05:22,  2.21it/s]

...

 97%|█████████▋| 1452/1500 [10:59<00:21,  2.21it/s]

cluster_labels 1452: 데이터가 부족하여 ARIMA 모델을 학습할 수 없습니다.


 98%|█████████▊| 1465/1500 [11:04<00:16,  2.16it/s]

cluster_labels 1465: 데이터가 부족하여 ARIMA 모델을 학습할 수 없습니다.


100%|██████████| 1500/1500 [11:19<00:00,  2.21it/s]


In [17]:
test_data = df[df['_type']=='test'].reset_index(drop=True).drop('mean_deposit_per_area_year', axis=1)

In [18]:
result_df['contract_year'] = result_df['contract_year'].astype(str)

test_data = test_data.merge(result_df, on=['cluster_labels', 'contract_year'], how='left')

In [19]:
df = pd.concat([train_data, test_data])

### `deposit_per_area` 예측

In [20]:
df['contract_year'] = df['contract_year'].astype(int)
df['contract_month'] = df['contract_month'].astype(int)

In [21]:
train_data = df[df['_type'] == 'train'].reset_index(drop=True)
test_data = df[df['_type'] == 'test'].reset_index(drop=True)

# 모든 컬럼 목록에서 '_type'만 제거
all_columns = df.columns.tolist()  # clusteredtest의 모든 컬럼 목록
all_columns.remove('_type')

all_columns_test = [col for col in all_columns if col not in ['deposit', 'deposit_per_area']]

In [22]:
# Holdout 데이터 설정
holdout_start = 202307
holdout_end = 202312
holdout_data = train_data[(train_data['contract_year_month'] >= holdout_start) & (train_data['contract_year_month'] <= holdout_end)]
train_data_tr = train_data[~((train_data['contract_year_month'] >= holdout_start) & (train_data['contract_year_month'] <= holdout_end))]

# Train/Test 데이터 분리
X_train = train_data_tr.drop(['deposit', 'deposit_per_area', '_type', 'index'], axis=1)
y_train = train_data_tr['deposit_per_area']
X_holdout = holdout_data.drop(['deposit', 'deposit_per_area', '_type', 'index'], axis=1)
y_holdout = holdout_data['deposit_per_area']
X_test = test_data.drop(['deposit', 'deposit_per_area', '_type', 'index'], axis=1).copy()
full_X = pd.concat([X_train, X_holdout])
full_y = pd.concat([y_train, y_holdout])
full_d = pd.concat([train_data, holdout_data])

In [23]:
%%time
lgb_params = {
    "boosting_type": "gbdt",
    "objective": "regression_l1",
    "metric": "mae",
    "max_depth": 30,
    "learning_rate": 0.15,
    "n_estimators": 200,
    "colsample_bytree": 0.8,
    # "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    # "l1_regularization": 0.1,
    # "l2_regularization": 10,
    "extra_trees": True,
    "num_leaves": 30
}

lgb_model = lgb.LGBMRegressor(**lgb_params)
lgb_model.fit(X_train, y_train)

CPU times: user 46.2 s, sys: 246 ms, total: 46.5 s
Wall time: 12.1 s


LGBMRegressor(colsample_bytree=0.8, extra_trees=True, learning_rate=0.15,
              max_depth=30, metric='mae', n_estimators=200, num_leaves=30,
              objective='regression_l1', random_state=42, verbose=-1)

In [24]:
y_pred = lgb_model.predict(X_holdout)
mae = mean_absolute_error(y_holdout, y_pred)
rmse = np.sqrt(mean_squared_error(y_holdout, y_pred))
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

MAE: 65.9169
RMSE: 96.8222


In [25]:
lgb_model.fit(full_X, full_y)
y_pred = lgb_model.predict(full_X)

lgb_test_pred = lgb_model.predict(X_test)

In [26]:
train_data['pred_deposit_per_area'] = y_pred
test_data['pred_deposit_per_area'] = lgb_test_pred

df = pd.concat([train_data, test_data])

In [27]:
df.shape

(1793006, 20)

### 이전 시점의 deposit 구하기

#### 직전값 매칭

In [28]:
# 1. 데이터 정렬
df = df.sort_values(by=['complex_id', 'area_m2', 'contract_year_month', 'contract_day'])

# 2. 그룹별 누적 최대값 계산
df['max_deposit_per_area'] = df.groupby(['complex_id', 'area_m2'])['deposit_per_area'].cummax()

# 3. 각 complex_id별 전체 max_deposit_per_area의 최대값 계산
complex_max_deposit = df.groupby('complex_id')['max_deposit_per_area'].max().reset_index()
complex_max_deposit.rename(columns={'max_deposit_per_area': 'complex_max_deposit_per_area'}, inplace=True)

# 4. 원본 데이터프레임에 병합
df = df.merge(complex_max_deposit, on='complex_id', how='left')

# 5. deposit_per_area가 결측치인 경우, complex_id별 최대값으로 채움
df['max_deposit_per_area'] = df['max_deposit_per_area'].fillna(df['complex_max_deposit_per_area'])

# 6. 불필요한 컬럼 제거
df.drop(columns=['complex_max_deposit_per_area'], inplace=True)

In [29]:
# 1. 데이터 정렬
df = df.sort_values(by=['complex_id', 'area_m2', 'contract_year_month', 'contract_day']).reset_index(drop=True)

# 2. 이전 deposit 값 가져오기
df['previous_deposit'] = df.groupby(['complex_id', 'area_m2'])['deposit'].shift(1)

# 3. 첫 번째 거래에서 previous_deposit가 NaN이고, deposit이 존재하는 경우 처리
df['transaction_order'] = df.groupby(['complex_id', 'area_m2']).cumcount()
mask = (df['previous_deposit'].isna()) & (df['transaction_order'] == 0) & (df['deposit'].notna())
df.loc[mask, 'previous_deposit'] = df.loc[mask, 'deposit']

# 4. remaining NaN 값에 대해 다른 area_m2의 deposit 값으로 채우기
# 4.1. 유효한 deposit 값 가져오기
valid_deposits = df[df['deposit'].notna()]
last_valid_deposits = valid_deposits.groupby(['complex_id', 'area_m2'])['deposit'].last().reset_index()

# 4.2. complex_id별로 area_m2와 deposit 값을 딕셔너리에 저장
complex_id_to_area_deposit = {}
for complex_id, group in last_valid_deposits.groupby('complex_id'):
    area_m2_values = group['area_m2'].values
    deposit_values = group['deposit'].values
    complex_id_to_area_deposit[complex_id] = (area_m2_values, deposit_values)

# 4.3. previous_deposit가 NaN인 행들 추출
missing_prev_deposit_idx = df[df['previous_deposit'].isna()].index
missing_prev_deposit_rows = df.loc[missing_prev_deposit_idx]

# 4.4. 각 complex_id별로 결측치 채우기
for complex_id, group in missing_prev_deposit_rows.groupby('complex_id'):
    if complex_id in complex_id_to_area_deposit:
        valid_area_m2_values, valid_deposit_values = complex_id_to_area_deposit[complex_id]
        missing_area_m2_values = group['area_m2'].values

        # numpy를 사용하여 가장 가까운 area_m2 찾기
        diff = np.abs(missing_area_m2_values[:, np.newaxis] - valid_area_m2_values[np.newaxis, :])
        min_idx = diff.argmin(axis=1)

        # 해당 deposit 값 가져오기
        deposit_values_to_fill = valid_deposit_values[min_idx]

        # previous_deposit 컬럼 업데이트
        df.loc[group.index, 'previous_deposit'] = deposit_values_to_fill

In [30]:
df.columns

Index(['index', 'area_m2', 'contract_year_month', 'contract_day',
       'contract_type', 'floor', 'built_year', 'latitude', 'longitude', 'age',
       'deposit', '_type', 'complex_id', 'max_deposit', 'cluster_labels',
       'deposit_per_area', 'contract_year', 'contract_month',
       'mean_deposit_per_area_year', 'pred_deposit_per_area',
       'max_deposit_per_area', 'previous_deposit', 'transaction_order'],
      dtype='object')

In [31]:
df = df.drop('transaction_order', axis=1)

### 반기별 max_deposit

In [32]:
# 1. 'contract_year_month'를 연도와 반기로 나누기
df['contract_year'] = df['contract_year_month'].astype(str).str[:4].astype(int)
df['contract_month'] = df['contract_year_month'].astype(str).str[4:].astype(int)
df['half_year'] = np.where(df['contract_month'] <= 6, 'H1', 'H2')

# 2. 반기별로 complex_id와 area_m2로 그룹화하여 deposit의 최대값을 구함
df['half_max_deposit'] = df.groupby(['complex_id', 'area_m2', 'contract_year', 'half_year'])['deposit'].transform('max')

# 3. 과거 데이터(2019~2023년)에서 (complex_id, area_m2)별 half_max_deposit의 최대값 추출
past_data = df[df['contract_year'] < 2024].copy()
max_deposit_past = past_data.groupby(['complex_id', 'area_m2'])['half_max_deposit'].max().reset_index()

# 4. 2024년 데이터에 과거의 최대 half_max_deposit 병합
data_2024 = df[df['contract_year'] == 2024].copy()
data_2024 = data_2024.merge(
    max_deposit_past,
    on=['complex_id', 'area_m2'],
    how='left',
    suffixes=('', '_max_past')
)

# half_max_deposit 업데이트
data_2024['half_max_deposit'] = data_2024['half_max_deposit_max_past']

# 5. half_max_deposit이 여전히 NaN인 경우 처리
missing_mask = data_2024['half_max_deposit'].isna()
if missing_mask.any():
    # 과거 데이터에서 complex_id별 area_m2와 half_max_deposit 목록 생성
    area_m2_past = past_data[['complex_id', 'area_m2']].drop_duplicates()
    area_m2_past = area_m2_past.merge(max_deposit_past, on=['complex_id', 'area_m2'], how='left')

    # complex_id별로 area_m2와 half_max_deposit를 딕셔너리에 저장
    complex_id_to_area_deposit = {}
    for complex_id, group in area_m2_past.groupby('complex_id'):
        area_m2_values = group['area_m2'].values
        half_max_deposit_values = group['half_max_deposit'].values
        complex_id_to_area_deposit[complex_id] = (area_m2_values, half_max_deposit_values)

    # 결측치가 있는 데이터에 대해 closest_area_m2의 half_max_deposit로 채움
    missing_data = data_2024[missing_mask].copy()

    def get_closest_half_max_deposit(row):
        complex_id = row['complex_id']
        area_m2 = row['area_m2']
        if complex_id in complex_id_to_area_deposit:
            areas, deposits = complex_id_to_area_deposit[complex_id]
            idx = np.abs(areas - area_m2).argmin()
            return deposits[idx]
        else:
            return np.nan

    missing_data['half_max_deposit'] = missing_data.apply(get_closest_half_max_deposit, axis=1)

    # 업데이트
    data_2024.loc[missing_data.index, 'half_max_deposit'] = missing_data['half_max_deposit']

df_no_2024 = df[df['contract_year'] < 2024]

# df_tmp_no_2024와 업데이트된 data_2024를 연결합니다.
df = pd.concat([df_no_2024, data_2024.drop('half_max_deposit_max_past', axis=1)], ignore_index=True)

# 필요에 따라 정렬합니다.
df = df.sort_values(by=['complex_id', 'area_m2', 'contract_year', 'half_year']).reset_index(drop=True)

### STD 할당

In [33]:
# 데이터 나누기
train_data = df[df['_type'] == 'train']
test_data = df[df['_type'] == 'test']

# train에서 deposit 값의 분산 계산
train_std = train_data.groupby(['complex_id', 'area_m2'])['deposit'].std().reset_index()
train_std.rename(columns={'deposit': 'deposit_std_id'}, inplace=True)

# train 데이터에 deposit_std_id 추가
train_data = train_data.merge(train_std, on=['complex_id', 'area_m2'], how='left')

# test 데이터에 train의 분산 값을 할당
test_data = test_data.merge(train_std, on=['complex_id', 'area_m2'], how='left', suffixes=('', '_train'))

# deposit_std_id가 결측치인 경우 처리
# 1. deposit_std_id가 NaN인 행들만 추출
missing_std = test_data[test_data['deposit_std_id'].isna()].copy()

# 2. complex_id별로 train_data의 area_m2와 deposit_std_id를 딕셔너리에 저장
complex_id_to_area_std = {}
for complex_id, group in train_data.groupby('complex_id'):
    area_m2_array = group['area_m2'].values
    deposit_std_id_array = group['deposit_std_id'].values
    complex_id_to_area_std[complex_id] = (area_m2_array, deposit_std_id_array)

# 3. 각 complex_id별로 결측치 채우기
for complex_id, group in missing_std.groupby('complex_id'):
    test_indices = group.index
    test_area_m2 = group['area_m2'].values

    # 해당 complex_id의 train_data가 있는 경우
    if complex_id in complex_id_to_area_std:
        train_area_m2, deposit_std_id_array = complex_id_to_area_std[complex_id]

        # numpy를 사용하여 가장 가까운 area_m2 찾기
        diff = np.abs(test_area_m2[:, np.newaxis] - train_area_m2[np.newaxis, :])
        min_idx = diff.argmin(axis=1)

        # 해당 deposit_std_id 값 가져오기
        closest_deposit_std_id = deposit_std_id_array[min_idx]

        # test_data의 deposit_std_id 업데이트
        test_data.loc[test_indices, 'deposit_std_id'] = closest_deposit_std_id
    else:
        # 해당 complex_id의 train_data가 없는 경우 그대로 NaN 유지
        pass

# train_data와 업데이트된 test_data를 결합
df = pd.concat([train_data, test_data], ignore_index=True)
df['deposit_std_id'] = df['deposit_std_id'].fillna(0)

In [34]:
# label_deposit 함수 정의
def label_deposit(value):
    if 0 <= value < 50000:
        return 0
    elif 50000 <= value < 100000:
        return 1
    elif 100000 <= value < 200000:
        return 2
    elif 200000 <= value < 500000:
        return 3
    else:
        return 4

df['pred_deposit'] = df['pred_deposit_per_area'] * df['area_m2']
# train 데이터의 pred_deposit 값을 label_deposit 함수로 변환하여 deposit_label에 할당
df['deposit_label'] = df['pred_deposit'].apply(label_deposit)

In [36]:
# df.to_csv(os.path.join(file_path, 'wonchan.csv'), index=False)

# FE 2

In [37]:
def categorize_area(x):
    range_start = (x // 50) * 50
    range_end = range_start + 49
    return f"{range_start}~{range_end}"

df['area_m2_category'] = df['area_m2'].apply(categorize_area)

# gangnam_lat = 37.498132408887
# gangnam_lon = 127.02839523744

# df['distance_from_gangnam'] = df.apply(lambda row: haversine((row['latitude'], row['longitude']), (gangnam_lat, gangnam_lon)), axis=1)

In [38]:
df = df.drop('distance_from_gangnam', axis=1)

In [40]:
%%time
subway = pd.read_csv(file_path + 'subwayInfo.csv')
school = pd.read_csv(file_path + 'schoolinfo.csv')
park = pd.read_csv(file_path + 'parkInfo.csv')

apt = df[['latitude', 'longitude']].drop_duplicates().reset_index(drop=True)


apt = apt.assign(
    # 1. Distance to the nearest subway
    nearest_subway_distance_km = feat_eng.nearest_POI(apt=apt[['latitude', 'longitude']], POI=subway)[0],

    # 2. Distance to the nearest (elementary, middel, high) school
    nearest_elementary_distance_km = feat_eng.nearest_POI(apt=apt[['latitude', 'longitude']], POI=school[school['schoolLevel']=='elementary'][['latitude', 'longitude']])[0],
    nearest_middle_distance_km = feat_eng.nearest_POI(apt=apt[['latitude', 'longitude']], POI=school[school['schoolLevel']=='middle'][['latitude', 'longitude']])[0],
    nearest_high_distance_km = feat_eng.nearest_POI(apt=apt[['latitude', 'longitude']], POI=school[school['schoolLevel']=='high'][['latitude', 'longitude']])[0],

    # 3. Distance to the nearest park & Area of the nearest park
    nearest_park_distance_km = feat_eng.nearest_POI(apt=apt[['latitude', 'longitude']], POI=park[['latitude', 'longitude']])[0],
    nearest_park_area = park['area'][feat_eng.nearest_POI(apt=apt[['latitude', 'longitude']], POI=park[['latitude', 'longitude']])[1]].values
)

apt = apt.assign(
    # 1. 지하철역
    num_subway_within_0_5 = feat_eng.count_within_radius(apt, subway, radius_km=0.5),
    num_subway_within_1 = feat_eng.count_within_radius(apt, subway, radius_km=1),
    num_subway_within_3 = feat_eng.count_within_radius(apt, subway, radius_km=2),

    # 2. 학교
    num_elementary_within_0_5 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='elementary'], radius_km=0.5),
    num_elementary_within_1 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='elementary'], radius_km=1),
    num_elementary_within_2 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='elementary'], radius_km=2),
    num_middle_within_0_5 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='middle'], radius_km=0.5),
    num_middle_within_1 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='middle'], radius_km=1),
    num_middle_within_2 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='middle'], radius_km=2),
    num_high_within_0_5 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='high'], radius_km=0.5),
    num_high_within_1 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='high'], radius_km=1),
    num_high_within_2 = feat_eng.count_within_radius(apt, school[school['schoolLevel']=='high'], radius_km=2),

    # 3. 공원
    num_park_within_0_8 = feat_eng.count_within_radius(apt, park, radius_km=0.8),
    num_park_within_1_5 = feat_eng.count_within_radius(apt, park, radius_km=1.5),
    num_park_within_2 = feat_eng.count_within_radius(apt, park, radius_km=2)
)

df = df.merge(apt, on=['latitude', 'longitude'], how='left')
del subway, school, park, apt
gc.collect()

CPU times: user 11.6 s, sys: 668 ms, total: 12.3 s
Wall time: 12.2 s


1503

In [41]:
df = df[~((df['_type'] == 'train') & (df['built_year'] == 2024))]

In [42]:
df['area_floor_interaction'] = df['area_m2'] * df['floor']

In [45]:
df.to_csv(os.path.join(file_path, 'wonchan.csv'), index=False)